In [ ]:
import numpy as np

def train_softmaxreg(X: np.ndarray, y: np.ndarray, learning_rate: float, iterations: int) -> tuple[list[float], ...]:
	# Your code here
    def _softmax(z):
        return np.exp(z) / np.sum(np.exp(z), axis=1, keepdims=True)
    
    def _to_one_hot(y):
        y_hot = np.zeros( (len(y), len(np.unique(y)) ) )
        for i in range(len(y)):
            y_hot[i, y[i]] = 1
        return y_hot
    
    def cross_entropy(y_pred, y_true):
        true_labels_idx = np.argmax(y_true, axis=1)
        return -np.sum(np.log(y_pred)[list(range(len(y_pred))),true_labels_idx])

    m, n = X.shape
    X = np.c_[np.ones(m), X]
    n += 1
    theta = np.zeros( (n, len(np.unique(y)) ) )
    y_hot = _to_one_hot(y)
    loss = []
    for _ in range(iterations):
        y_pred = _softmax(X @ theta)
        gradients = X.T @ (y_pred - y_hot)
        theta -= learning_rate * gradients
        loss.append(np.round(cross_entropy(y_pred, y_hot), 4))
    return np.round(theta.T, 4), loss


In [56]:
train_softmaxreg(
    np.array([[0.5, -1.2], [-0.3, 1.1], [0.8, -0.6]]), 
    np.array([0, 1, 2]), 
    0.01,
    10
)

(array([[-0.0011,  0.0145, -0.0921],
        [ 0.002 , -0.0598,  0.1263],
        [-0.0009,  0.0453, -0.0342]]),
 [np.float64(3.2958),
  np.float64(3.2611),
  np.float64(3.2272),
  np.float64(3.1941),
  np.float64(3.1618),
  np.float64(3.1302),
  np.float64(3.0993),
  np.float64(3.0692),
  np.float64(3.0398),
  np.float64(3.011)])

In [ ]:
import numpy as np
import scipy.stats as stats

def analyze_ab_test(control_outcomes: list, treatment_outcomes: list, confidence_level: float = 0.95, min_detectable_effect: float = 0.02) -> dict:

    # Checking if any group is empty or not
    if not control_outcomes or not treatment_outcomes:
        return {}

    # Changing to numpy arrays for broadcast and calculating sample sizes
    control_outcomes = np.array(control_outcomes)
    treatment_outcomes = np.array(treatment_outcomes)
    n1 = control_outcomes.size
    n2 = treatment_outcomes.size

    # Calculating statistics for further calculations
    control_count = (control_outcomes == 1).sum()
    treatment_count = (treatment_outcomes == 1).sum()

    control_rate = control_count / n1
    treatment_rate = treatment_count / n2

    absolute_lift = treatment_rate - control_rate
    relative_lift = absolute_lift / control_rate

    # Calculating z-statistic and p-value
    p = (control_count + treatment_count) / (n1 + n2)
    z_statistic = (treatment_rate - control_rate) / (np.sqrt(p * (1 - p) * (1/n1 + 1/n2)))
    p_value = stats.norm.sf(abs(z_statistic)) * 2

    # Confidence interval
    z_alpha = stats.norm.ppf(1 - (1 - confidence_level)/2)
    se = np.sqrt(control_rate * (1 - control_rate) / n1 + treatment_rate * (1 - treatment_rate) / n2)
    confidence_interval = (treatment_rate - control_rate - se * z_alpha, treatment_rate - control_rate + se * z_alpha)

    # Calculating significance
    statistical_significance = p_value < (1 - confidence_level)
    practical_significance = abs(absolute_lift) >= min_detectable_effect

    # Sample size and recommendation
    z_beta = stats.norm.ppf(0.8)
    required_sample_size = (2 * (z_alpha + z_beta) ** 2  * p * (1 - p)) / min_detectable_effect

    if statistical_significance and practical_significance and absolute_lift > 0:
        recommendation = 'launch_treatment'
    elif statistical_significance and (not practical_significance or absolute_lift < 0):
        recommendation = 'keep_control'
    else:
        recommendation = 'continue_testing'

    return {
        'control_rate': control_rate,
        'treatment_rate': treatment_rate,
        'absolute_lift': round(absolute_lift, 2),
        'relative_lift': relative_lift,
        'z_statistic': round(z_statistic, 4),
        'p_value': p_value,
        'confidence_interval': confidence_interval,
        'statistically_significant': statistical_significance,
        'practically_significant': practical_significance,
        'required_sample_size': required_sample_size,
        'recommendation': recommendation
    }


In [45]:
control_outcomes = [1,1,1,0,0,0,1,0,1,0]*50
treatment_outcomes = [1,1,1,1,0,0,1,0,1,0]*50
confidence_level = 0.95
min_detectable_effect = 0.02
analyze_ab_test(control_outcomes, treatment_outcomes, confidence_level, min_detectable_effect)

{'control_rate': np.float64(0.5),
 'treatment_rate': np.float64(0.6),
 'absolute_lift': np.float64(0.09999999999999998),
 'relative_lift': np.float64(0.55),
 'z-statistic': np.float64(3.17820863081864),
 'p-value': np.float64(0.0014818807747203912),
 'confidence-interval': (np.float64(0.16135657784036067),
  np.float64(0.03864342215963927))}